In [60]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import common as cm

# Part 1: Information Gain

Important note: this exercise uses Pandas (for data manipulation and analysis) and Graphviz (for graph-drawing) libraries. 

This exercise consists of 3 parts. Complete the first part to get a mark of 3.0, the first two parts to get 4.0, complete all assignments to get 5.0. 

1.1 ) There are 10 objects (data) characterized with 5 binary attributes:

In [61]:
attributeNames = ["attr 1", "attr 2", "attr 3", "attr 4", "attr 5"]

data = pd.DataFrame(
    [
        [1, 0, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [0, 1, 1, 1, 1],
        [1, 0, 1, 0, 1],
        [1, 0, 0, 1, 1],
        [0, 0, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 1, 1],
        [0, 1, 0, 0, 1],
        [0, 0, 0, 1, 1],
    ],
    columns=attributeNames,
)

1.2) Each object is assigned to either a class "0" or "1". The assignments are as follows (cl):

In [62]:
data["cl"] = [1, 1, 1, 0, 0, 1, 1, 1, 0, 0]

Hint: How one can read data (columns) in Pandas:

In [63]:
print(data["cl"])
print(list(data["cl"]))
print(set(data["cl"]))
print(data["attr 1"])

0    1
1    1
2    1
3    0
4    0
5    1
6    1
7    1
8    0
9    0
Name: cl, dtype: int64
[1, 1, 1, 0, 0, 1, 1, 1, 0, 0]
{0, 1}
0    1
1    1
2    0
3    1
4    1
5    0
6    1
7    1
8    0
9    0
Name: attr 1, dtype: int64


Hint: How split data (Pandas DataFrame) base on column:

In [64]:
data[data['cl']==0]



,attr 1,attr 2,attr 3,attr 4,attr 5,cl
3,1,0,1,0,1,0
4,1,0,0,1,1,0
8,0,1,0,0,1,0
9,0,0,0,1,1,0


In [65]:
data[data['cl']==1]

,attr 1,attr 2,attr 3,attr 4,attr 5,cl
0,1,0,1,1,1,1
1,1,1,0,0,1,1
2,0,1,1,1,1,1
5,0,0,1,1,1,1
6,1,1,1,1,1,1
7,1,0,0,1,1,1


Hint: How take values from column (Pandas Series):

In [66]:
for id, row in data['cl'].items():
    print(id,row)

0 1
1 1
2 1
3 0
4 0
5 1
6 1
7 1
8 0
9 0


1.3 )  Finish the below function for calculating entropy. $H(CL) = - \sum_{y \in CL}p(y)log_2p(y)$ It should return a value of entropy for an input vector CL. Assume that $log_2(0)$ is equal to 0.

In [67]:
def getnumber(cl):
    p0 = 0
    p1 = 0
    for id, row in cl.items():
        if row == 0:
            p0 += 1
        else:
            p1 += 1
#     print(p0,p1)
    return p0,p1

def getEntropy(cl,p0,p1):
    if isinstance(cl, int):
        length = cl
    else:
        length = len(cl)
#     print(cl)
    if length == 0:
        return 0
    if p0/length == 0 :
        return -((p0/length)*0)-((p1/length)*math.log2(p1/length))
    if p1/length == 0:
         return -((p0/length)*math.log2(p0/length))-((p1/length)*0)
    return -((p0/length)*math.log2(p0/length))-((p1/length)*math.log2(p1/length))

1.4 ) Calculate the entropy for the CL vector:

In [68]:
p0,p1 = getnumber(data["cl"])
print(getEntropy(data["cl"],p0,p1))


0.9709505944546686


1.5) Finish the below function for calculating a conditional entropy: $H(CL|X) = - \sum_{x \in X} \sum_{y \in CL} p(x,y) log_2 \frac{p(x,y)}{p(x)}$. Assume that $log_2(0)$ is equal to 0 and if $p(x) = 0$, $\frac{p(x,y)}{p(x)}$ is equal to 0 as well.

In [69]:
def getConditionalEntropy(cl, attr):
    counter = []
    matrix = [[0 for i in range(2)] for i in range(2)]
    for clas,attribute in zip(cl.items(),attr.items()):
        if clas[1] == 0 and attribute[1] ==0:
            matrix[0][0] += 1
        if clas[1] == 0 and attribute[1] ==1:
            matrix[1][0] +=1
        if clas[1] == 1 and attribute[1] ==0:
            matrix[0][1] +=1
        if clas[1] == 1 and attribute[1] ==1:
            matrix[1][1] +=1
#     print(matrix)
    for i in matrix:
#         print(i[0],i[1])
        counter.append(getEntropy(sum(i),i[0],i[1]))
#     print(counter)
    return (sum(matrix[0])/len(cl))*counter[0] + (sum(matrix[1])/len(cl))*counter[1]
    

1.6 ) Calculate conditional entropies for given attribiutes.

In [70]:
print(getConditionalEntropy(data["cl"], data["attr 1"]))
print(getConditionalEntropy(data["cl"], data["attr 5"]))

0.9509775004326937
0.9709505944546686


1.8) Finish the below function for calculating information gain:

In [71]:
def getInformationGain(cl, attr):
    p0,p1 = getnumber(cl)
    entropy1 = getEntropy(cl,p0,p1)
    entropy2 = getConditionalEntropy(cl, attr)
    return entropy1 - entropy2

In [72]:
print(getInformationGain(data["cl"], data["attr 1"]))
print(getInformationGain(data["cl"], data["attr 5"]))

0.01997309402197489
0.0


# Part 2: ID3 algorithm

Decision tree consists of decision nodes and leaves. Nodes split data while leaves classify objects. Consider the class "Node" provided below. It consists of 4 fields:
- attr - attribute ID (use the names in attributeNames vector)
- left - left branch, i.e., a reference to other node
- right - right branch, i.e., a reference to other node
- value - a decision. If node = None, then the node is not a leaf. If value is not None, then a node is considered a leaf. 

Method __call__ returns the decision if the node is a leaf (i.e., when value is not None). 
Otherwise, it calls either the left or the right branch of an input object, based on the attribute value (0 -> left children; 1 -> right children). In this way, we can traverse the decision tree in order to find the final decision.

In [73]:
class Node:
    def __init__(self, attr, left, right, value):
        self.attr = attr
        self.left = left
        self.right = right
        self.value = value

    def __call__(self, obj):
        if self.value is None:
            if obj[self.attr] == 0:
                return self.left(obj)
            else:
                return self.right(obj)
        else:
            return self.value
        
### EXAMPLE
def example(obj):
    root = Node(0, None, None, None)
    lChildren = Node(1, None, None, None)
    rChildren = Node(None, None, None, 2)
    root.left = lChildren
    root.right = rChildren
    llChildren = Node(None, None, None, 3)
    rrChildren = Node(None, None, None, 4)
    lChildren.left = llChildren
    lChildren.right = rrChildren
    print(root(obj))
    
example([0, 0])
example([0, 1])
example([1, 0])
example([1, 1])

3
4
2
2


2.1) Create an initial root. Set the value (decision) to 1. 

In [74]:
init_root = Node(None,None,None,1)

2.2) Use a getErrorRate method in common.py auxiliary file to calculate the error rate. The decision is made based on the majority rule. In case of tie, the method takes 0 as the default class.

In [75]:
cm.getErrorRate(init_root,data)

0.4

2.3) Use printGraph method (see the common.py file) to draw the decision tree and save it in a png file.

In [76]:
cm.printGraph(init_root)

2.4) Calculate information gain for all attribiutes.

In [82]:
def InfoGain(data,attributes):
    infoGain = []
    for i in range(len(attributes)):
        infoGain.append(getInformationGain(data["cl"], data[attributeNames[i]]))
    return infoGain
print(InfoGain(data,attributeNames))

[0.01997309402197489, 0.0464393446710154, 0.12451124978365313, 0.0912774462416801, 0.0]


2.5) Choose the best attribute to split the data. Construct two new nodes: one for $x_i$ = 0 decision and the second for $x_i$ = 1; connect them with the root (left and right branch). Remember to update the root. 

In [91]:
bestattr = InfoGain(data,attributeNames).index(max(InfoGain(data,attributeNames)))
print(bestattr)
init_root = Node(bestattr,None,None,None)
left_node = Node(None,None,None,0)
right_node = Node(None,None,None,1)
init_root.left = left_node
init_root.right = right_node

2


2.6) Print the graph and calculate the error rate. What happened with the error rate?

In [86]:
cm.printGraph(init_root)
cm.getErrorRate(init_root,data)

0.30000000000000004

2.7) Split the 'data' (table) based on the selected attribiute, i.e., create two new tables.

In [92]:
left_data = data[data[attributeNames[attributeNames[bestattr]]] == 0]
right_data = data[data[attributeNames[attributeNames[bestattr]]] == 1]

TypeError: list indices must be integers or slices, not str

2.8) Let us start with the left node. Firstly, calculate information gain for this node.

In [81]:
left_gain = InfoGain(left_data,attributeNames)
print(left_gain)

NameError: name 'left_data' is not defined

2.9) Choose the best attribute to split the data and then update the decision tree.

In [22]:
bestattr = left_gain.index(max(left_gain))
left_node = Node(bestattr,None,None,None)
l_left_node = Node(None,None,None,0)
r_left_node = Node(None,None,None,1)
left_node.left = l_left_node
left_node.right = r_left_node
init_root.left = left_node

NameError: name 'left_gain' is not defined

2.10) Print the graph and calculate the error rate. What happened with the error rate?

In [23]:
cm.printGraph(init_root)

2.11) Split data (remember that we split left_data, not data).

In [24]:
leftLeft_data = left_data[left_data[attributeNames[bestattr]] == 0]
leftRight_data = left_data[left_data[attributeNames[bestattr]] == 1]

NameError: name 'left_data' is not defined

2.12) Repeat the whole process for the right node.

In [25]:
right_gain = InfoGain(right_data,attributeNames)
print(right_gain)

NameError: name 'right_data' is not defined

In [26]:
bestattr = right_gain.index(max(right_gain))
right_node = Node(bestattr,None,None,None)
l_right_node = Node(None,None,None,0)
r_right_node = Node(None,None,None,1)
right_node.left = l_right_node
right_node.right = r_right_node
init_root.right = right_node

NameError: name 'right_gain' is not defined

In [27]:
cm.printGraph(init_root)
cm.getErrorRate(init_root,data)

0.30000000000000004

In [28]:
rightLeft_data = right_data[right_data[attributeNames[bestattr]] == 0]
rightRight_data = right_data[right_data[attributeNames[bestattr]] == 1]

NameError: name 'right_data' is not defined

2.13) Let's consider left-left node. Calculate information gain for it.

In [29]:
left_left_gain = InfoGain(leftLeft_data,attributeNames)
print(left_left_gain)

NameError: name 'leftLeft_data' is not defined

2.14) Will adding a new node to the tree improve its effectiveness? Why? Why not?

In [30]:
### No, becaouse we've reached the node(leaf) with entropy equal to 0 - for every record in left_left_data the class outcome is the same

2.15) Calculate information gain for the left-right node.

In [31]:
left_right_gain = InfoGain(leftRight_data,attributeNames)
print(left_right_gain)

NameError: name 'leftRight_data' is not defined

In [32]:
bestattr = left_right_gain.index(max(left_right_gain))
l_right_node = Node(bestattr,None,None,None)
l_l_right_node = Node(None,None,None,0)
r_l_right_node = Node(None,None,None,1)
l_right_node.left = l_l_right_node
l_right_node.right = r_l_right_node
right_node.left = l_right_node
init_root.right = right_node

NameError: name 'left_right_gain' is not defined

In [33]:
cm.printGraph(init_root)
cm.getErrorRate(init_root,data)

0.30000000000000004

2.16) What happened with the error rate? Is it necessary to keep these two newly added leaves?

In [34]:
### The error rate is the same, so we may assume that these newly added leaves didn't make our decision tree any better

2.17) Finish creating the right side of the tree

In [35]:
r_right_gain = InfoGain(rightRight_data,attributeNames)
print(r_right_gain)

NameError: name 'rightRight_data' is not defined

In [36]:
### we can clearly see that creating the tree any further would be an overfitting, so we can stop right now, we can't lower the error rate anyway.

# Part 3: automated construction of decision trees

3.1 Complete the following function for automated construct of decision trees, so that it returns a decision tree for the given data and attribute list. Note that this is a recusive method, i.e., calls itself.

In [37]:
max_depth = 0

def createTree(data, attributeNames, depth=0):
    data = data.reset_index().drop("index", axis=1)
    ### TODO

3.2) Build a decision tree for a training dataset in the common.py auxiliary file, for diffrent values of max_depth.  Calculate & compare the error rates for training and validation datasets.

In [ ]:
max_depth = 10

In [ ]:
# Training dataset
train_attributeNames, train_data = cm.getTrainingDataSet()
### TODO

In [ ]:
# Validation dataset
valid_attributesName, valid_data = cm.getValidationDataSet()
### TODO

3.3) Consider only the training data set and answer the following questions:
* What is the miximum depth of the tree (consider only the training data set)?
* The tree building process should stop when there is no improvement in error rate (why?). Check for which value of "max_dept" there is no improvement in error rate. 

In [ ]:
for i in range(10):
    max_depth = i
    ### TODO